In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_ingressos = pd.DataFrame()

# importando notas de 2009 a 2017
for i in range(2009,2018):
    print(i)
    df_ingresso = pd.read_csv('../data_science/discentes/discentes-'+str(i)+'.csv',sep=';')
    df_ingressos = pd.concat([df_ingressos, df_ingresso])
    

2009
2010
2011
2012
2013
2014
2015
2016
2017


In [3]:
# curso CeT
id_curso = 10320810.0
# filtrando igressantes de CeT
df_ingressoCeT = df_ingressos[df_ingressos["id_curso"] == id_curso]

In [4]:
df_relacao = pd.read_csv('../data_science/matriculas-2013-2017.csv', sep=';')
cols = ['id_discente', 'nome', 'cotista', 'nota_ingresso']
df_relacao = df_relacao[cols]

In [5]:
df_merge = pd.merge(df_ingressoCeT, df_relacao, left_on='nome_discente', right_on='nome')
colunas = ['matricula', 'id_discente', 'ano_ingresso', 'periodo_ingresso',
           'forma_ingresso', 'tipo_discente', 'status', 'cotista', 'nota_ingresso']

df_merge = df_merge[colunas]

In [6]:
df_merge['periodo_ingresso'] = df_merge.periodo_ingresso.astype(int)

In [7]:
df_matriculas = pd.DataFrame()

for i in range(11,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('../data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])
        
df_matriculas.columns

20111
20112
20121
20122
20131
20132
20141
20142
20151


/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


20152
20161
20162
20171
20172


Index(['Unnamed: 10', 'descricao', 'discente', 'faltas_unidade', 'id_curso',
       'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao',
       'unidade'],
      dtype='object')

In [8]:
df_matriculas = df_matriculas[df_matriculas.id_curso == float(id_curso)]

In [9]:
df_turmas = pd.read_csv('../data_science/turmas-2010-2017-ok.csv')
df_matTurma = pd.merge(df_matriculas, df_turmas, on='id_turma')

/home/andressa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
colunas = ['discente', 'ano', 'periodo']
df_discentePer = df_matTurma[colunas]
df_discentePer['anoperiodoTer'] = (df_discentePer['ano'].map(str)+df_discentePer['periodo'].map(str))

/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
#pegar o ultimo ano/periodo que pagaram disciplina
series_maxPer = df_discentePer.groupby('discente').anoperiodoTer.max().reset_index()

# Ingressantes
df_ingressantePeriodo = pd.merge(df_merge, series_maxPer, left_on='id_discente',
                                 right_on='discente')
df_ingressantePeriodo['anoperiodoIngresso'] = (df_ingressantePeriodo['ano_ingresso'].map(str)+
                                               df_ingressantePeriodo['periodo_ingresso'].map(str))
df_tableIngre = df_ingressantePeriodo.anoperiodoIngresso.value_counts().reset_index()

# Desistentes: CANCELADO, TRANCADO
df_desis = df_ingressantePeriodo[(df_ingressantePeriodo['status'] == 'CANCELADO') |
                                 (df_ingressantePeriodo['status'] == 'TRANCADO')]
df_tableDesis = df_desis.anoperiodoTer.value_counts().reset_index()

# Formados: CONCLUÍDO
df_conc = df_ingressantePeriodo[df_ingressantePeriodo['status'] == 'CONCLUÍDO']
df_tableConc = df_conc.anoperiodoTer.value_counts().reset_index()

In [16]:
df_tableDesis.columns

Index(['index', 'anoperiodoTer'], dtype='object')

In [17]:
lista = df_tableDesis.anoperiodoTer.unique()
df_todosperiodos = pd.DataFrame()
df_todosperiodos['periodos'] = [20092,20101,20102,20111,20112,20121,20122,20131,20132,20141,20142,
                                20151,20152,20161,20162,20171,20172]
teste = df_todosperiodos[~df_todosperiodos.periodos.isin(lista)]
teste

periodos
0      20092
1      20101
2      20102
3      20111
4      20112
5      20121
6      20122
7      20131
8      20132
9      20141
10     20142
11     20151
12     20152
13     20161
14     20162
15     20171
16     20172

In [18]:
df_todosperiodos = pd.DataFrame()
df_todosperiodos['anoperiodo'] = [20092,20101,20102,20111,20112,20121,20122,20131,20132,20141,20142,
                                20151,20152,20161,20162,20171,20172]
col = ['anoperiodo','quantidade']

################################

df_tableIngre['quantidade'] = df_tableIngre['anoperiodoIngresso']
df_tableIngre['anoperiodo'] = df_tableIngre['index']
df_tableIngre = df_tableIngre[col]
df_tableIngre['anoperiodo'] = df_tableIngre.anoperiodo.astype(int)

lista = df_tableIngre.anoperiodo.unique()
faltaPeriodos = df_todosperiodos[~df_todosperiodos.anoperiodo.isin(lista)]

df_tableIngre = pd.concat([df_tableIngre, faltaPeriodos])
df_tableIngre = df_tableIngre.fillna(0)
df_tableIngre['quantidade'] = df_tableIngre.quantidade.astype(int)
df_tableIngre = df_tableIngre.sort_values('anoperiodo')

###################################

df_tableDesis['quantidade'] = df_tableDesis['anoperiodoTer']
df_tableDesis['anoperiodo'] = df_tableDesis['index']
df_tableDesis = df_tableDesis[col]
df_tableDesis['anoperiodo'] = df_tableDesis.anoperiodo.astype(int)

lista = df_tableDesis.anoperiodo.unique()
faltaPeriodos = df_todosperiodos[~df_todosperiodos.anoperiodo.isin(lista)]

df_tableDesis = pd.concat([df_tableDesis, faltaPeriodos])
df_tableDesis = df_tableDesis.fillna(0)
df_tableDesis['quantidade'] = df_tableDesis.quantidade.astype(int)
df_tableDesis = df_tableDesis.sort_values('anoperiodo')

####################################

df_tableConc['quantidade'] = df_tableConc['anoperiodoTer']
df_tableConc['anoperiodo'] = df_tableConc['index']
df_tableConc = df_tableConc[col]
df_tableConc['anoperiodo'] = df_tableConc.anoperiodo.astype(int)

lista = df_tableConc.anoperiodo.unique()
faltaPeriodos = df_todosperiodos[~df_todosperiodos.anoperiodo.isin(lista)]

df_tableConc = pd.concat([df_tableConc, faltaPeriodos])
df_tableConc = df_tableConc.fillna(0)
df_tableConc['quantidade'] = df_tableConc.quantidade.astype(int)
df_tableConc = df_tableConc.sort_values('anoperiodo')

/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  app.launch_new_instance()
/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and si

In [19]:
df_tableIngre.to_csv('../data_science/ingressantesQnt.csv')
df_tableDesis.to_csv('../data_science/desistentesQnt.csv')
df_tableConc.to_csv('../data_science/concluintesQnt.csv')